# Variational Information Bottleneck
https://arxiv.org/abs/1612.00410

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 100
epochs = 200
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda:1"
else:
    device = "cpu"

In [2]:
root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1)*2-1)]) # [-1,1]
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

In [3]:
from pixyz.distributions import Normal, Categorical
from pixyz.models import Model
from pixyz.losses import KullbackLeibler, Parameter
from torch.optim.lr_scheduler import StepLR
from pixyz.utils import print_latex

In [4]:
x_dim = 784
y_dim = 10
z_dim = 256


# encoder model p(z|x)
class Encoder(Normal):
    def __init__(self):
        super().__init__(cond_var=["x"], var=["z"], name="p")

        self.fc1 = nn.Linear(x_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc31 = nn.Linear(1024, z_dim)
        self.fc32 = nn.Linear(1024, z_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {"loc": self.fc31(h),
                "scale": F.softplus(self.fc32(h)-5.0)} # see apendix

    
# classifier model q(y|z)    
class Classifier(Categorical):
    def __init__(self):
        super().__init__(cond_var=["z"], var=["y"], name="q")

        self.fc1 = nn.Linear(z_dim, y_dim)

    def forward(self, z):
        return {"probs": F.softmax(self.fc1(z), dim=1)}
    

p = Encoder().to(device)
q = Classifier().to(device)

# prior model r(z)
prior = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.),
               var=["z"], features_shape=[z_dim], name="r").to(device)

In [5]:
print(p)
print_latex(p)

Distribution:
  p(z|x)
Network architecture:
  Encoder(
    name=p, distribution_name=Normal,
    var=['z'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=784, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=1024, bias=True)
    (fc31): Linear(in_features=1024, out_features=256, bias=True)
    (fc32): Linear(in_features=1024, out_features=256, bias=True)
  )


<IPython.core.display.Math object>

In [6]:
print(q)
print_latex(q)

Distribution:
  q(y|z)
Network architecture:
  Classifier(
    name=q, distribution_name=Categorical,
    var=['y'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
    (fc1): Linear(in_features=256, out_features=10, bias=True)
  )


<IPython.core.display.Math object>

In [7]:
kl = KullbackLeibler(p, prior)
exp = -q.log_prob().expectation(p)
lmb = Parameter("lmb")

loss = (exp + lmb * kl).mean()
print(loss)
print_latex(loss)

mean \left(D_{KL} \left[p(z|x)||r(z) \right] lmb - \mathbb{E}_{p(z|x)} \left[\log q(y|z) \right] \right)


<IPython.core.display.Math object>

In [8]:
model = Model(loss, distributions=[p, q],
              optimizer=optim.Adam, optimizer_params={"lr":1e-4, "betas":(0.5, 0.999)})
print(model)
print_latex(model)

Distributions (for training): 
  p(z|x), q(y|z) 
Loss function: 
  mean \left(D_{KL} \left[p(z|x)||r(z) \right] lmb - \mathbb{E}_{p(z|x)} \left[\log q(y|z) \right] \right) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.5, 0.999)
      eps: 1e-08
      lr: 0.0001
      weight_decay: 0
  )


<IPython.core.display.Math object>

In [9]:
# Learning Rate Schedule
scheduler = StepLR(model.optimizer, step_size=2, gamma=0.97)

In [10]:
def train(epoch):
    scheduler.step()
    train_loss = 0
    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = torch.eye(10)[y].to(device)
        loss = model.train({"x": x, "y": y, "lmb": 1e-3})
        train_loss += loss
        
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    
    return train_loss

In [11]:
def test(epoch):
    test_loss = 0
    correct = 0
    total = 0    
    for x, y in tqdm(test_loader):
        x = x.to(device)
        y = torch.eye(10)[y].to(device)        
        loss = model.test({"x": x, "y": y, "lmb": 1e-3})
        test_loss += loss
        
        z = p.sample_mean({"x": x})
        pred_y = q.sample_mean({"z": z})
        total += y.size(0)
        correct += (pred_y.argmax(dim=1) == y.argmax(dim=1)).sum().item()      

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    test_accuracy = 100 * correct / total
    print('Test loss: {:.4f}, Test accuracy: {:.4f}'.format(test_loss, test_accuracy))
    return test_loss, test_accuracy

In [12]:
writer = SummaryWriter()

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss, test_accuracy = test(epoch)
    
    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)
    writer.add_scalar('test_accuracy', test_accuracy, epoch)        
    
writer.close()

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 Train loss: 0.6362


  0%|          | 0/600 [00:00<?, ?it/s]

Test loss: 0.3440, Test accuracy: 93.4700


  0%|          | 0/100 [00:00<?, ?it/s]

...

Test loss: 0.0967, Test accuracy: 98.6900


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 200 Train loss: 0.0174


100%|██████████| 100/100 [00:01<00:00, 63.20it/s]

Test loss: 0.0972, Test accuracy: 98.7200
